In [3]:
import csv
import numpy as np
from matplotlib import pyplot
from sklearn import linear_model
from sklearn.model_selection import cross_validate
import pandas as pd
import datetime as dt
import math
import sys

In [47]:
#Sort the data in the file
#read a file
file = open('Fukadani201607to201711.txt')
Allfile = file.readlines()
file.close()
Allfilelisted = [0]*len(Allfile)

#split each list at blanks
for n in range(len(Allfile)):
    Allfilelisted[n] = Allfile[n].split()
    
#extract title
Data = Allfilelisted[4:]
Datatitle = Allfilelisted[0][:]


In [48]:
# print(Data)
print(Datatitle)

['TIMESTAMP', 'RECORD', 'Bat_Avg', 'Pit_WL_Avg', 'L_UR_Avg', 'L_UL_Avg', 'L_DR_Avg', 'L_DL_Avg', 'Load_Avg', 'Load_Avg_difference', 'WL_up_Avg', 'WL_cha_Avg', 'turb1000_Avg', 'turb4000_Avg', 'DiffVolt_Avg', 'hp_Tot(1)', 'hp_Tot(2)', 'hp_Tot(3)', 'hp_Tot(4)', 'hp_Tot(5)', 'hp_Tot(6)', 'vel_P_Tot', 'WL_FMR_Avg', 'hp_Tot(7)', 'hp_Tot(8)', 'hp_Tot(9)', 'hp_Tot(10)']


In [49]:
#Start using pandas
df = pd.DataFrame(Data, dtype=np.float64)
#Put date and time together because I wanna let "to_datetime" read date and time later.
date_time = df[:][0] + " " + df[:][1]
#Let "to_datatime" read the date and time and convert them into datetime64
converted_time = pd.to_datetime(date_time)
# print(converted_time)
#insert this date&time data into the Data set(Delite the original date and time(Column 0 & 1))
df1 = pd.concat([converted_time, df.iloc[:, 2:]], axis=1, ignore_index=False)
#Put titles on the columns
df1.columns = Datatitle
print(df1)

                 TIMESTAMP    RECORD  Bat_Avg  Pit_WL_Avg  L_UR_Avg  L_UL_Avg  \
0      2016-06-01 00:03:00  369472.0    12.66      77.070    336.40    274.80   
1      2016-06-01 00:04:00  369473.0    12.66      77.070    336.40    274.80   
2      2016-06-01 00:05:00  369474.0    12.66      77.090    336.30    274.70   
3      2016-06-01 00:06:00  369475.0    12.66      77.080    336.50    274.80   
4      2016-06-01 00:07:00  369476.0    12.66      77.060    336.50    274.90   
5      2016-06-01 00:08:00  369477.0    12.66      77.080    336.40    274.80   
6      2016-06-01 00:09:00  369478.0    12.66      77.100    336.40    274.80   
7      2016-06-01 00:10:00  369479.0    12.66      77.090    336.40    274.80   
8      2016-06-01 00:11:00  369480.0    12.66      77.070    336.40    274.80   
9      2016-06-01 00:12:00  369481.0    12.66      77.080    336.40    274.80   
10     2016-06-01 00:13:00  369482.0    12.66      77.100    336.40    274.80   
11     2016-06-01 00:14:00  

In [51]:
print(df1[671875:671877])
print(df1.dtypes)

                 TIMESTAMP  RECORD  Bat_Avg  Pit_WL_Avg  L_UR_Avg  L_UL_Avg  \
671875 2017-09-11 13:32:00     0.0      NaN         NaN       NaN       NaN   
671876 2017-09-11 13:33:00     0.0      NaN         NaN       NaN       NaN   

        L_DR_Avg  L_DL_Avg  Load_Avg  Load_Avg_difference     ...      \
671875       NaN       NaN       NaN                  NaN     ...       
671876       NaN       NaN       NaN                  NaN     ...       

        hp_Tot(3)  hp_Tot(4)  hp_Tot(5)  hp_Tot(6)  vel_P_Tot  WL_FMR_Avg  \
671875        NaN        NaN        NaN        NaN        NaN         NaN   
671876        NaN        NaN        NaN        NaN        NaN         NaN   

        hp_Tot(7)  hp_Tot(8)  hp_Tot(9)  hp_Tot(10)  
671875        NaN        NaN        NaN         NaN  
671876        NaN        NaN        NaN         NaN  

[2 rows x 27 columns]
TIMESTAMP              datetime64[ns]
RECORD                        float64
Bat_Avg                       float64
Pit_WL_Avg 

In [52]:
# print(df1.set_index('TIMESTAMP')[df1.set_index('TIMESTAMP').index.duplicated(keep=False)])
# df[df.index.duplicated()]
df_exact = df1[df1['TIMESTAMP'] == '2017-09-23 13:11:00']
print(df_exact)



                 TIMESTAMP    RECORD  Bat_Avg  Pit_WL_Avg  L_UR_Avg  L_UL_Avg  \
689074 2017-09-23 13:11:00  591775.0    12.71       77.52     240.6     188.4   

        L_DR_Avg  L_DL_Avg  Load_Avg  Load_Avg_difference     ...      \
689074     177.5     266.2     873.0                  1.0     ...       

        hp_Tot(3)  hp_Tot(4)  hp_Tot(5)  hp_Tot(6)  vel_P_Tot  WL_FMR_Avg  \
689074       57.0       25.0        7.0        3.0      2.044       4.298   

        hp_Tot(7)  hp_Tot(8)  hp_Tot(9)  hp_Tot(10)  
689074        2.0        0.0        0.0         0.0  

[1 rows x 27 columns]


In [54]:
# Fill out missing date&time
df2 = df1.set_index('TIMESTAMP').asfreq('T').reset_index()
#Fill out NaNs with 0.
# df3 = df2.fillna(0)
print(df2)

                 TIMESTAMP    RECORD  Bat_Avg  Pit_WL_Avg  L_UR_Avg  L_UL_Avg  \
0      2016-06-01 00:03:00  369472.0    12.66      77.070    336.40    274.80   
1      2016-06-01 00:04:00  369473.0    12.66      77.070    336.40    274.80   
2      2016-06-01 00:05:00  369474.0    12.66      77.090    336.30    274.70   
3      2016-06-01 00:06:00  369475.0    12.66      77.080    336.50    274.80   
4      2016-06-01 00:07:00  369476.0    12.66      77.060    336.50    274.90   
5      2016-06-01 00:08:00  369477.0    12.66      77.080    336.40    274.80   
6      2016-06-01 00:09:00  369478.0    12.66      77.100    336.40    274.80   
7      2016-06-01 00:10:00  369479.0    12.66      77.090    336.40    274.80   
8      2016-06-01 00:11:00  369480.0    12.66      77.070    336.40    274.80   
9      2016-06-01 00:12:00  369481.0    12.66      77.080    336.40    274.80   
10     2016-06-01 00:13:00  369482.0    12.66      77.100    336.40    274.80   
11     2016-06-01 00:14:00  

In [55]:
#Save data
df2.to_csv(path_or_buf="Fukadani201607to201711.csv")

In [20]:
# df4 = df3.set_index('TIMESTAMP').resample('10T').mean().reset_index()
# print(df4)


                TIMESTAMP    RECORD    Bat_Avg  outC_VR_Avg  VR_Tot(1)  \
0     2018-03-20 15:30:00      35.0  12.072857     0.305429   2.714286   
1     2018-03-20 15:40:00      43.5  12.070000     0.300800   0.000000   
2     2018-03-20 15:50:00      53.5  12.070000     0.309100   0.000000   
3     2018-03-20 16:00:00      63.5  12.041000     0.309200   0.400000   
4     2018-03-20 16:10:00      73.5  12.040000     0.307100   0.000000   
5     2018-03-20 16:20:00      83.5  12.040000     0.306700   0.000000   
6     2018-03-20 16:30:00      93.5  12.040000     0.307600   0.000000   
7     2018-03-20 16:40:00     103.5  12.040000     0.305900   0.000000   
8     2018-03-20 16:50:00     113.5  12.040000     0.308500   0.000000   
9     2018-03-20 17:00:00     123.5  12.040000     0.297300   0.000000   
10    2018-03-20 17:10:00     133.5  12.040000     0.300100   0.000000   
11    2018-03-20 17:20:00     143.5  12.040000     0.294100   0.000000   
12    2018-03-20 17:30:00     153.5  1

In [21]:
# #Import Rainfall, WL, Velocity data from another file.
# #Planning to get this data and "df4" together.

# #Sort the data in the file
# #read a file
# fileR = open('Rainfall_WL_Velocity_2018.txt')
# AllfileR = fileR.readlines()
# file.close()
# AllfilelistedR = [0]*len(AllfileR)

# #split each list at blanks
# for n in range(len(AllfileR)):
#     AllfilelistedR[n] = AllfileR[n].split()

# DataR = AllfilelistedR[1:]
# DatatitleR = AllfilelistedR[0][:]


In [22]:
# #Start using pandas
# dfR = pd.DataFrame(DataR, dtype=np.float64)
# #Put date and time together because I wanna let "to_datetime" read date and time later.
# date_timeR = dfR[:][0] + " " + dfR[:][1]
# #Let "to_datatime" read the date and time and convert them into datetime64
# converted_timeR = pd.to_datetime(date_timeR)
# # print(converted_time)
# #insert this date&time data into the Data set(Delite the original date and time(Column 0 & 1))
# df1R = pd.concat([converted_timeR, dfR.iloc[:, 2:]], axis=1)
# #Put titles on the columns
# df1R.columns = ["TIME","Rainfall","Velocity","WL"]
# print(df1R)

                     TIME  Rainfall  Velocity   WL
0     2018-01-01 00:00:00       0.0      0.65  1.4
1     2018-01-01 00:10:00       0.0      0.65  1.4
2     2018-01-01 00:20:00       0.0      0.65  1.3
3     2018-01-01 00:30:00       0.0      0.65  1.3
4     2018-01-01 00:40:00       0.0      0.65  1.4
5     2018-01-01 00:50:00       0.0      0.65  1.3
6     2018-01-01 01:00:00       0.0      0.65  1.3
7     2018-01-01 01:10:00       0.0      0.65  1.4
8     2018-01-01 01:20:00       0.0      0.65  1.4
9     2018-01-01 01:30:00       0.0      0.65  1.4
10    2018-01-01 01:40:00       0.0      0.65  1.3
11    2018-01-01 01:50:00       0.0      0.65  1.4
12    2018-01-01 02:00:00       0.0      0.65  1.4
13    2018-01-01 02:10:00       0.0      0.65  1.3
14    2018-01-01 02:20:00       0.0      0.65  1.3
15    2018-01-01 02:30:00       0.0      0.65  1.3
16    2018-01-01 02:40:00       0.0      0.65  1.3
17    2018-01-01 02:50:00       0.0      0.65  1.3
18    2018-01-01 03:00:00      

In [23]:
# # Fill out missing date&time with the mean values of each column.(Notice!! I haven't checked the filled out data. You should check out if they are filled out correctly)
# df2R = df1R.set_index('TIME').asfreq('10T', fill_value= 0).reset_index()

In [24]:
# #Output the date of the first and the last data
# #And then put cut out df1R data by using these date
# print(df4["TIMESTAMP"].head(1))
# print(df4["TIMESTAMP"].tail(1))

# print(df1R[df2R['TIME'] == '2018-03-20 15:30:00'])
# print(df1R[df2R['TIME'] == '2018-12-16 16:30:00'])


# df3R = df2R[11325:50355+1].reset_index(drop=True)
# print(df3R)

0   2018-03-20 15:30:00
Name: TIMESTAMP, dtype: datetime64[ns]
39030   2018-12-16 16:30:00
Name: TIMESTAMP, dtype: datetime64[ns]
                     TIME  Rainfall  Velocity   WL
11325 2018-03-20 15:30:00       0.0      0.65  0.8
                     TIME  Rainfall  Velocity   WL
50355 2018-12-18 18:20:00       0.0      1.97  4.9
                     TIME  Rainfall  Velocity   WL
0     2018-03-20 15:30:00       0.0      0.65  0.8
1     2018-03-20 15:40:00       0.0      0.65  0.9
2     2018-03-20 15:50:00       0.0      0.65  0.9
3     2018-03-20 16:00:00       0.0      0.65  0.9
4     2018-03-20 16:10:00       0.0      0.65  0.9
5     2018-03-20 16:20:00       0.0      0.65  0.9
6     2018-03-20 16:30:00       0.0      0.65  0.9
7     2018-03-20 16:40:00       0.0      0.65  0.9
8     2018-03-20 16:50:00       0.0      0.65  0.8
9     2018-03-20 17:00:00       0.0      0.65  0.8
10    2018-03-20 17:10:00       0.0      0.65  0.8
11    2018-03-20 17:20:00       0.0      0.65  0.8
12 

C:\Users\yutak\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
C:\Users\yutak\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  import sys


In [25]:
# df_fin = pd.concat([df4, df3R.iloc[:, 1:]], axis=1)
# print(df_fin)

                TIMESTAMP    RECORD    Bat_Avg  outC_VR_Avg  VR_Tot(1)  \
0     2018-03-20 15:30:00      35.0  12.072857     0.305429   2.714286   
1     2018-03-20 15:40:00      43.5  12.070000     0.300800   0.000000   
2     2018-03-20 15:50:00      53.5  12.070000     0.309100   0.000000   
3     2018-03-20 16:00:00      63.5  12.041000     0.309200   0.400000   
4     2018-03-20 16:10:00      73.5  12.040000     0.307100   0.000000   
5     2018-03-20 16:20:00      83.5  12.040000     0.306700   0.000000   
6     2018-03-20 16:30:00      93.5  12.040000     0.307600   0.000000   
7     2018-03-20 16:40:00     103.5  12.040000     0.305900   0.000000   
8     2018-03-20 16:50:00     113.5  12.040000     0.308500   0.000000   
9     2018-03-20 17:00:00     123.5  12.040000     0.297300   0.000000   
10    2018-03-20 17:10:00     133.5  12.040000     0.300100   0.000000   
11    2018-03-20 17:20:00     143.5  12.040000     0.294100   0.000000   
12    2018-03-20 17:30:00     153.5  1

In [26]:
# #Save data
# df_fin.to_csv(path_or_buf="logger3_2018-03-20_2018-12-16.csv")